In [9]:
# imports 
import pandas as pd
import oracledb

In [30]:
def sql(sql, is_select=False):
    with oracledb.connect(user='jisel', password='12341234', dsn='localhost/ucentral') as conexion:
        with conexion.cursor() as cursor:
            ejecucion = cursor.execute(sql)
            if is_select:
                columns = [col[0] for col in cursor.description]
                cursor.rowfactory = lambda *args: dict(zip(columns, args))
                return cursor.fetchall()
            else:
                conexion.commit()


In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/DiegoGamba3420/Proyecto-bases-de-datos/main/Encuesta_Pobreza.csv')

In [4]:
data.head()

,Unnamed: 0,nombre_departamento,nombre_municipio,nombre_lugar_poblado,cod_familia,fecha_nacimiento,sexo,nucleo,estado_conyugal,discapacidad_permanente,sabe_leer_escribir,asistencia_centro_educativo,razon_dejo_asistir,nivel_inscrito,nivel_aprobado,categoria_ocupacional,grupo_etnico,es_jefe_familia,edad
0,0,SAN MARCOS,SAN PABLO,BARRANCA HONDA,609497670.0,1962-11-27,Mujer,1.0,Casado,NaN,No,No,No le interesa,Ninguno,Ninguno,Trabajador(a) cuenta propia agricola,No indigena,Si,60
1,1,SAN MARCOS,COMITANCILLO,CHAMAQUE,8600484.0,2003-02-05,Mujer,1.0,Soltero(a),NaN,Si,Si,NaN,Sexto Primaria,Quinto Primaria,NaN,Mam,No,19
2,2,SAN MARCOS,SAN LORENZO,SANTA ROSA (CASERIO),13648135.0,1977-07-30,Mujer,1.0,Casado,NaN,Si,No,Falta de dinero,Ninguno,Primero Primaria,No Trabaja,Mam,No,45
3,3,SAN MARCOS,COMITANCILLO,CHAMAQUE,8600465.0,2004-04-26,Mujer,1.0,Soltero(a),NaN,Si,Si,NaN,Cuarto Primaria,Tercero Primaria,NaN,Mam,No,18
4,4,SAN MARCOS,SAN PABLO,LA CUMBRE,609499178.0,2006-04-29,Hombre,1.0,Soltero(a),NaN,Si,Si,NaN,Quinto Primaria,Cuarto Primaria,NaN,No indigena,No,16


In [44]:
# Se importan departamentos

departamentos_unicos = set()
for departamento in data['nombre_departamento']:
    departamentos_unicos.add(departamento)

sql("DELETE FROM departamento")
for departamento in departamentos_unicos:
    sql(f"INSERT INTO departamento (nombre) VALUES ('{departamento}')")

departamentos = sql("SELECT * FROM departamento", is_select=True)
departamentos_ids = {}
for departamento in departamentos:
    departamentos_ids[departamento['NOMBRE']] = departamento['ID']

print(departamentos_ids)

{'QUICHE': 2, 'HUEHUETENANGO': 1, 'SAN MARCOS': 3, 'SOLOLA': 4, 'TOTONICAPAN': 5}


In [45]:
# Se importan municipios

municipios_unicos = set()

for i, municipio in enumerate(data['nombre_municipio']):
    departamento_id = departamentos_ids[data['nombre_departamento'][i]]
    municipios_unicos.add((departamento_id, municipio))

sql("DELETE FROM municipio")
for departamento_id, municipio in municipios_unicos:
    sql(f"INSERT INTO municipio (nombre, departamento_id) VALUES ('{municipio}', {departamento_id})")

municipios_ids = {}
municipios = sql("SELECT * FROM municipio", is_select=True)
for municipio in municipios:
    municipios_ids[(municipio['NOMBRE'], municipio['DEPARTAMENTO_ID'])] = municipio['ID']

print(municipios_ids)

{('SANTA MARIA CHIQUIMULA', 5): 2, ('COMITANCILLO', 3): 1, ('SAN ANTONIO PALOPO', 4): 3, ('LA REFORMA', 3): 4, ('SANTA CRUZ LA LAGUNA', 4): 5, ('NENTON', 1): 6, ('CONCEPCION HUISTA', 1): 7, ('SAN PEDRO SACATEPEQUEZ', 3): 8, ('SANTA BARBARA', 1): 9, ('SANTA LUCIA LA REFORMA', 5): 10, ('TAJUMULCO', 3): 11, ('SAN PABLO', 3): 12, ('SAN JUAN ATITAN', 1): 13, ('SAN SEBASTIAN HUEHUETENANGO', 1): 14, ('EL QUETZAL', 3): 15, ('NUEVO PROGRESO', 3): 16, ('SAN ANTONIO SACATEPEQUEZ', 3): 17, ('NAHUALA', 4): 18, ('SAN PEDRO NECTA', 1): 19, ('SAN CRISTOBAL TOTONICAPAN', 5): 20, ('SAN MARCOS', 3): 21, ('SAN JUAN LA LAGUNA', 4): 22, ('SACAPULAS', 2): 23, ('TACANA', 3): 24, ('SAN FRANCISCO EL ALTO', 5): 25, ('TOTONICAPAN', 5): 26, ('SANTA CATARINA IXTAHUACAN', 4): 27, ('MOMOSTENANGO', 5): 28, ('LA DEMOCRACIA', 1): 29, ('SAN RAFAEL PIE DE LA CUESTA', 3): 30, ('HUEHUETENANGO', 1): 31, ('SANTA CRUZ DEL QUICHE', 2): 32, ('CHIANTLA', 1): 33, ('AYUTLA', 3): 34, ('SAN BARTOLO', 5): 35, ('LA LIBERTAD', 1): 36, (

In [48]:
# Se importan poblados

poblados_unicos = set()

for i, poblado in enumerate(data['nombre_lugar_poblado']):
    departamento_id = departamentos_ids[data['nombre_departamento'][i]]
    municipio_id = municipios_ids[(data['nombre_municipio'][i], departamento_id)]
    poblados_unicos.add((municipio_id, poblado))

sql("DELETE FROM poblado")
for municipio_id, poblado in poblados_unicos:
    sql(f"INSERT INTO poblado (nombre, municipio_id) VALUES ('{poblado}', {municipio_id})")

poblados_ids = {}
poblados = sql("SELECT * FROM poblado", is_select=True)
for poblado in poblados:
    poblados_ids[(poblado['NOMBRE'], poblado['MUNICIPIO_ID'])] = poblado['ID']
print(poblados_ids)

{('LA CUMBRE', 54): 227, ('PERICONCITO (CASERIO)', 55): 228, ('XEJUYUP (CASERIO)', 10): 131, ('SAN GABRIEL (CASERIO)', 3): 132, ('SANTO DOMINGO', 12): 133, ('PACHIPAC', 18): 134, ('SAN FRANCISCO SOCHE', 8): 135, ('PANYEVAR', 22): 136, ('SAN FRANCISCO', 48): 137, ('EL PLAN', 40): 138, ('LA CUMBRE', 12): 139, ('CHICOL', 9): 140, ('CIENAGA', 20): 141, ('XOLSACMALJA (CASERIO)', 26): 142, ('PALESTINA', 22): 143, ('TABLON VIEJO', 59): 144, ('EL TESORO', 23): 145, ('SAN RAMON (CIUDAD)', 20): 146, ('XANTUN (CASERIO)', 26): 147, ('BARRIO SANTA ISABEL', 28): 148, ('CERRO DE ORO', 49): 149, ('POJOPOM', 53): 150, ('PARRAXCHAJ (CIUDAD)', 35): 151, ('NINCHIN', 47): 152, ('IXQUILAM', 13): 153, ('JULEN (CASERIO)', 42): 154, ('XEJUYUB', 2): 155, ('LA FELICIDAD', 17): 156, ('LOS CHURUNELES II (CASERIO)', 48): 157, ('LA FEDERACION', 21): 158, ('CHOMOXOL', 28): 159, ('LOS CEREZOS', 42): 160, ('GUALTUX (CIUDAD)', 10): 161, ('EL CARRIZAL', 31): 162, ('PUEBLO VIEJO', 52): 163, ('CUNLAJ', 24): 164, ('CHOBAQUI

In [52]:
# Se insertan familias

familia_unicos = set()

for i, codigo in enumerate(data['cod_familia']):
    departamento_id = departamentos_ids[data['nombre_departamento'][i]]
    municipio_id = municipios_ids[(data['nombre_municipio'][i], departamento_id)]
    poblado_id = poblados_ids[(data['nombre_lugar_poblado'][i], municipio_id)]
    nucleo = data['nucleo'][i]
    familia_unicos.add((poblado_id, codigo, nucleo))

sql("DELETE FROM familia")
for poblado_id, codigo, nucleo in familia_unicos:
    sql(f"INSERT INTO familia (codigo, nucleo, poblado_id) VALUES ('{codigo}', '{nucleo}', {poblado_id})")

familias_ids = {}
familias = sql("SELECT * FROM familia", is_select=True)
for familia in familias:
    familias_ids[(familia['CODIGO'], familia['POBLADO_ID'])] = familia['ID']
print(familias_ids)

{('27797.0', 149): 571, ('7697988.0', 136): 587, ('9300325.0', 170): 637, ('27911.0', 149): 668, ('9249118.0', 181): 696, ('603498497.0', 164): 697, ('9347997.0', 142): 750, ('8797893.0', 169): 754, ('9248386.0', 141): 539, ('8797904.0', 169): 569, ('8600484.0', 168): 764, ('11652265.0', 134): 765, ('8797923.0', 169): 783, ('605498507.0', 177): 534, ('12701.0', 229): 535, ('606998951.0', 193): 536, ('11648418.0', 178): 537, ('607504066.0', 252): 538, ('350497927.0', 237): 540, ('709498650.0', 228): 541, ('659997746.0', 249): 542, ('604998325.0', 173): 543, ('17736.0', 254): 544, ('8550515.0', 152): 545, ('9300697.0', 170): 546, ('652997902.0', 129): 547, ('10649123.0', 140): 548, ('611498024.0', 138): 549, ('402997814.0', 155): 550, ('359497970.0', 149): 551, ('356997680.0', 195): 552, ('27791.0', 149): 553, ('611497818.0', 183): 554, ('9248528.0', 242): 555, ('17804.0', 229): 556, ('13647872.0', 171): 557, ('8797798.0', 151): 558, ('700502551.0', 214): 559, ('13599642.0', 244): 560, (

In [56]:
# Se importan estados conyugales

estados_unicos = set()
for estado in data['estado_conyugal']:
    estados_unicos.add(estado)

sql("DELETE FROM estado_conyugal")
for estado in estados_unicos:
    sql(f"INSERT INTO estado_conyugal (nombre) VALUES ('{estado}')")

estados = sql("SELECT * FROM estado_conyugal", is_select=True)
estados_ids = {}
for estado in estados:
    estados_ids[estado['NOMBRE']] = estado['ID']

print(estados_ids)

{'Unido': 2, 'Soltero(a)': 1, 'Separado(a) de union': 3, 'Separado(a) de matrimonio': 4, 'Viudo(a)': 5, 'Casado': 6, 'Menor de edad': 7}


In [57]:
# Se importan los grupos étnicos

grupos_unicos = set()
for grupo in data['grupo_etnico']:
    grupos_unicos.add(grupo)

sql("DELETE FROM grupo_etnico")
for grupo in grupos_unicos:
    sql(f"INSERT INTO grupo_etnico (nombre) VALUES ('{grupo}')")

grupos = sql("SELECT * FROM grupo_etnico", is_select=True)
grupos_ids = {}
for grupo in grupos:
    grupos_ids[grupo['NOMBRE']] = grupo['ID']

print(grupos_ids)

{'Akateko': 2, 'Kiche': 1, 'No indigena': 3, 'Mam': 4, 'Tzutujil': 5, 'Jacalteco': 6, 'Kaqchikel': 7, 'Qanjobal': 8, 'Chuj': 9}


In [77]:
# Se importan las personas

sql("DELETE FROM persona")

for i, codigo in enumerate(data['cod_familia']):
    departamento_id = departamentos_ids[data['nombre_departamento'][i]]
    municipio_id = municipios_ids[(data['nombre_municipio'][i], departamento_id)]
    poblado_id = poblados_ids[(data['nombre_lugar_poblado'][i], municipio_id)]
    
    familia_id = familias_ids[(str(codigo), poblado_id)]
    estado_conyugal_id = estados_ids[data['estado_conyugal'][i]]
    grupo_etnico_id = grupos_ids[data['grupo_etnico'][i]]
    fecha = data['fecha_nacimiento'][i]
    sexo = data['sexo'][i]
    discapacidad = data['discapacidad_permanente'][i]
    razon_desercion = data['razon_dejo_asistir'][i]
    sabe_leer = 'T' if data['sabe_leer_escribir'][i] == 'Si' else 'F'
    es_jefe_familiar = 'T' if data['es_jefe_familia'][i] == 'Si' else 'F'
    categoria_ocupacional = data['categoria_ocupacional'][i]
    asistencia_centro_educativo = data['asistencia_centro_educativo'][i]
    edad = int(data['edad'][i])
    nivel_inscrito = data['nivel_inscrito'][i]
    nivel_aprobado = data['nivel_aprobado'][i]
    sql(f"""
        INSERT INTO persona (
            familia_id,
            estado_conyugal_id,
            grupo_etnico_id,
            fecha,
            sexo,
            discapacidad,
            razon_desercion,
            sabe_leer,
            es_jefe_familiar,
            categoria_ocupacional,
            asistencia_centro_educativo,
            edad,
            nivel_inscrito,
            nivel_aprobado
        ) VALUES(
            {familia_id},
            {estado_conyugal_id},
            {grupo_etnico_id},
            TO_DATE('{fecha}', 'yyyy-mm-dd'),
            '{sexo}',
            '{discapacidad}',
            '{razon_desercion}',
            '{sabe_leer}',
            '{es_jefe_familiar}',
            '{categoria_ocupacional}',
            '{asistencia_centro_educativo}',
            {edad},
            '{nivel_inscrito}',
            '{nivel_aprobado}'
        )
    """)

personas = sql("SELECT * FROM persona", is_select=True)
for persona in personas:
    print(persona)

{'ID': 1805, 'DISCAPACIDAD': 'nan', 'RAZON_DESERCION': 'nan', 'SABE_LEER': 'T', 'CATEGORIA_OCUPACIONAL': 'nan', 'ASISTENCIA_CENTRO_EDUCATIVO': 'Si', 'EDAD': 18, 'FECHA': datetime.datetime(2004, 2, 12, 0, 0), 'NIVEL_APROBADO': 'Quinto Primaria', 'NIVEL_INSCRITO': 'Sexto Primaria', 'ES_JEFE_FAMILIAR': 'F', 'SEXO': 'Mujer', 'FAMILIA_ID': 548, 'ESTADO_CONYUGAL_ID': 1, 'GRUPO_ETNICO_ID': 4}
{'ID': 1806, 'DISCAPACIDAD': 'nan', 'RAZON_DESERCION': 'nan', 'SABE_LEER': 'F', 'CATEGORIA_OCUPACIONAL': 'nan', 'ASISTENCIA_CENTRO_EDUCATIVO': 'nan', 'EDAD': 9, 'FECHA': datetime.datetime(2013, 11, 1, 0, 0), 'NIVEL_APROBADO': 'Ninguno', 'NIVEL_INSCRITO': 'Ninguno', 'ES_JEFE_FAMILIAR': 'F', 'SEXO': 'Hombre', 'FAMILIA_ID': 603, 'ESTADO_CONYUGAL_ID': 7, 'GRUPO_ETNICO_ID': 1}
{'ID': 1807, 'DISCAPACIDAD': 'nan', 'RAZON_DESERCION': 'nan', 'SABE_LEER': 'T', 'CATEGORIA_OCUPACIONAL': 'nan', 'ASISTENCIA_CENTRO_EDUCATIVO': 'Si', 'EDAD': 12, 'FECHA': datetime.datetime(2010, 4, 13, 0, 0), 'NIVEL_APROBADO': 'Preprimar